In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir

In [45]:
df_append = pd.DataFrame()
i = 1
for filename in listdir("dataset\\dataset\\train\\boxes_transcripts_labels\\"):
# for filename in listdir("dataset\\dataset\\train\\New\\"):
    df_temp = pd.read_csv("dataset\\dataset\\train\\boxes_transcripts_labels\\" + filename, sep = ",", header = None)
    df_temp.columns = ['start_index', 'end_index', 'x_top_left', 'y_top_left', 'x_bottom_right', 'y_bottom_right','transcript','field']
    df_temp_trimmed = df_temp.loc[df_temp['field'] != 'OTHER']
    df_temp_trimmed.insert(0, 'File No.', i)
    i = i + 1
    df_append = pd.concat([df_append, df_temp_trimmed], ignore_index = True)

df_append['x_center'] = df_append[['x_top_left', 'x_bottom_right']].mean(axis=1)
df_append['y_center'] = df_append[['y_top_left', 'y_bottom_right']].mean(axis=1)

df_append.drop(['x_top_left', 'y_top_left','x_bottom_right','y_bottom_right'], axis=1)
df_append = df_append[['File No.','start_index', 'end_index', 'x_center', 'y_center','transcript','field']]

#df_append.columns = ['start_index', 'end_index', 'x_top_left', 'y_top_left', 'x_bottom_right', 'y_bottom_right','transcript','field']


In [27]:
# df_box1 = df_append.loc[df_append['field'] == 'box1WagesTipsAndOtherCompensations']
# mask = df_box1['transcript'].isin([161950.28])

,start_index,end_index,x_top_left,y_top_left,x_bottom_right,y_bottom_right,transcript,field


In [56]:
df_append.loc[df_append['File No.'] == 24]

,File No.,start_index,end_index,x_center,y_center,transcript,field
629,24,90,98,753.5,356.0,161590.28,box1WagesTipsAndOtherCompensations
630,24,148,155,1794.0,356.0,41434.61,box2FederalIncomeTaxWithheld
631,24,251,259,753.5,513.5,150840.93,box3SocialSecurityWages
632,24,309,316,1794.0,513.5,11539.33,box4SocialSecurityTaxWithheld
633,24,737,742,749.0,998.5,Porter,employerName
634,24,744,746,941.5,998.5,Ltd,employerName
635,24,748,750,1081.5,998.5,Inc,employerName
636,24,766,768,696.5,1076.5,229,employerAddressStreet_name
637,24,770,776,906.5,1076.5,Jeffrey,employerAddressStreet_name
638,24,778,783,1169.0,1076.5,Summit,employerAddressStreet_name
